# This Notebook is an example of using pcap pandas analytics to build a temporal graph from Ian Taylor's Newt orchestration of Hamlet by Scenes

In [15]:
import IPython
import matplotlib
from distutils.version import StrictVersion
ipy4 = False
violin = False
if StrictVersion(IPython.__version__) >= StrictVersion("4.0"):
    ipy4 = True
if StrictVersion(matplotlib.__version__) >= StrictVersion("1.5"):
    violin = True
    
import log2pd as lpd
#import plotpd as ppd
import pandas as pd
import log2plot as l2plot
import analystPlots as appd
import logAnalyticsLib as lalib
import matplotlib.pyplot as plt
import matplotlib.cm as cmx
import numpy as np
import os
import warnings
from netaddr import IPNetwork
import glob
from StringIO import StringIO
import fileinput
from pandas.lib import Timestamp
import datetime
import subprocess
#from IPython.display import Image
#os.system("taskset -p 0xFFFFFFFF %d" % os.getpid())
# Setup some plot defaults for the duration of this notebook
#ppd.setup_plot_defaults()
warnings.filterwarnings("ignore")
root_dir = '.'
%matplotlib inline

In [16]:
#df = lalib.load_hdf5('./store-mgen-mcast-ham.hdf5')  
from scipy.interpolate import interp1d
from operator import itemgetter

In [17]:
import json

In [18]:
json_data=open("./data/core_actor_mapping.json").read()
d = json.loads(json_data)

In [19]:
# Read the node file and create a dict of ip address to CORE nodename
n_to_a_dict={}
for actor,value in d.iteritems():
    n_to_a_dict[value['address']]=value['node']
print n_to_a_dict
#n_to_a_dict['224.0.5.2']='n50'
#
# Use CORE names for graph nodes

import networkx as nx
G=nx.DiGraph()
nodes = n_to_a_dict.values()
G.add_nodes_from(nodes)
n_weights={node:0 for node in nodes}
print n_weights
nx.set_node_attributes(G,'weight',n_weights)
print G.nodes(data=True)

{u'10.0.0.33': u'n33', u'10.0.0.32': u'n32', u'10.0.0.31': u'n31', u'10.0.0.30': u'n30', u'10.0.0.34': u'n34', u'10.0.0.15': u'n15', u'10.0.0.14': u'n14', u'10.0.0.17': u'n17', u'10.0.0.16': u'n16', u'10.0.0.11': u'n11', u'10.0.0.10': u'n10', u'10.0.0.13': u'n13', u'10.0.0.12': u'n12', u'10.0.0.19': u'n19', u'10.0.0.18': u'n18', u'10.0.0.9': u'n9', u'10.0.0.8': u'n8', u'10.0.0.5': u'n5', u'10.0.0.4': u'n4', u'10.0.0.7': u'n7', u'10.0.0.6': u'n6', u'10.0.0.1': u'n1', u'10.0.0.3': u'n3', u'10.0.0.2': u'n2', u'10.0.0.28': u'n28', u'10.0.0.29': u'n29', u'10.0.0.24': u'n24', u'10.0.0.25': u'n25', u'10.0.0.26': u'n26', u'10.0.0.27': u'n27', u'10.0.0.20': u'n20', u'10.0.0.21': u'n21', u'10.0.0.22': u'n22', u'10.0.0.23': u'n23'}
{u'n12': 0, u'n13': 0, u'n10': 0, u'n11': 0, u'n16': 0, u'n17': 0, u'n14': 0, u'n15': 0, u'n18': 0, u'n19': 0, u'n30': 0, u'n31': 0, u'n32': 0, u'n33': 0, u'n34': 0, u'n29': 0, u'n28': 0, u'n23': 0, u'n22': 0, u'n21': 0, u'n20': 0, u'n27': 0, u'n26': 0, u'n25': 0, u'n2

In [20]:
node_to_actor={}
for actor,value in d.iteritems():
    node_to_actor[value['node']]=actor

In [21]:
H=G.copy()
result='./results/Collect_ActIIISceneIII/'
scenepd=lpd.load_pcap_data(result,
                           duplicates=True,
                           timeseries=True,
                           dataSet=True)
sorteddf=scenepd.set_index('time')
sorteddf=sorteddf.sort_index()
sorteddf=sorteddf[(sorteddf["frameType"]=="UDP") & (sorteddf["dir"]==2)]

print sorteddf['ipSrc']
###### Build and estimated  CAG from adjacencies
last_speaker=None
# Build a graph from multicast packets received
# Use ipSrc information to create pair-wise adjacencies
for index,row in sorteddf.iterrows():
    if last_speaker == None:
        last_speaker = row.ipSrc
    elif last_speaker == row.ipSrc:
        pass
    else:
        H.node[n_to_a_dict[row.ipSrc]]["weight"] += 1
        if H.has_edge(n_to_a_dict[last_speaker],n_to_a_dict[row.ipSrc]):
            H[n_to_a_dict[last_speaker]][n_to_a_dict[row.ipSrc]]['weight'] += 1
        else:
            H.add_edge(n_to_a_dict[last_speaker],n_to_a_dict[row.ipSrc],weight= 1)
        last_speaker= row.ipSrc

load_pcap_data() Error: pcap nodups file in dataset, file ./results/Collect_ActIIISceneIII//n4/n4.eth0.pcap.nodups not processed.
load_pcap_data() Error: pcap nodups file in dataset, file ./results/Collect_ActIIISceneIII//n30/n30.eth0.pcap.nodups not processed.
load_pcap_data() Error: pcap nodups file in dataset, file ./results/Collect_ActIIISceneIII//n20/n20.eth0.pcap.nodups not processed.
load_pcap_data() Error: pcap nodups file in dataset, file ./results/Collect_ActIIISceneIII//n33/n33.eth0.pcap.nodups not processed.
load_pcap_data() Error: pcap nodups file in dataset, file ./results/Collect_ActIIISceneIII//n29/n29.eth0.pcap.nodups not processed.
load_pcap_data() Error: pcap nodups file in dataset, file ./results/Collect_ActIIISceneIII//n14/n14.eth0.pcap.nodups not processed.
load_pcap_data() Error: pcap nodups file in dataset, file ./results/Collect_ActIIISceneIII//n32/n32.eth0.pcap.nodups not processed.
load_pcap_data() Error: pcap nodups file in dataset, file ./results/Collect_Ac

In [25]:
with open('hamlet_stanzas.json', 'r') as handle:
    stanza_dict = json.load(handle)

In [54]:
actor1="Osric"
actor2="Hamlet"
count=0
first=True
for act in stanza_dict.iteritems():
    for scene in act[1].iteritems():
        stanzas=scene[1]
        if first:
            first=False
        for stanza in stanzas.iteritems():
            if stanza[1]['actor']==actor1 and stanza[1]['next_actor']==actor2:
                count=count+1
print count

18
